In [ ]:
## Reference: https://www.youtube.com/watch?v=xekqR10lQNo
## For personal study purposes only

In [1]:
from dowhy import datasets

import pandas as pd
import numpy as np

from causalinference import CausalModel

In [2]:
data = datasets.linear_dataset(
    beta=10,
    num_common_causes=4,
    num_samples=10_000,
    treatment_is_binary=True,
    outcome_is_binary=False,
)

df = data['df']
df = df.rename({'v0': 'treatment', 'y': 'outcome'}, axis=1)
df['treatment'] = df['treatment'].astype(int)

In [3]:
causal = CausalModel(
    Y=df['outcome'].values,
    D=df['treatment'].values,
    X=df[['W0', 'W1', 'W2', 'W3']].values,
)

print(causal.summary_stats)


Summary Statistics

                      Controls (N_c=3391)        Treated (N_t=6609)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y       -2.725        5.172       18.035        5.828       20.760

                      Controls (N_c=3391)        Treated (N_t=6609)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0       -0.609        0.756        0.637        0.826        1.573
             X1        0.340        1.002        0.466        1.001        0.126
             X2        0.318        1.006        0.447        1.005        0.129
             X3       -0.233        0.893        0.645        0.922        0.967



In [5]:
causal.est_propensity_s()
print(causal.propensity)


Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept     -0.065      0.051     -1.283      0.200     -0.165      0.034
            X0      3.850      0.090     42.608      0.000      3.673      4.028
            X3      2.671      0.070     38.347      0.000      2.535      2.808
            X2      0.485      0.038     12.713      0.000      0.410      0.560
            X1      0.376      0.039      9.651      0.000      0.300      0.452
         X3*X1     -0.116      0.044     -2.623      0.009     -0.202     -0.029
         X0*X0      0.109      0.064      1.697      0.090     -0.017      0.235



## weights:

* units with treatment: weight = 1/propensity
* units without treatment: weight = 1/(1-propensity)

In [7]:
causal.est_via_weighting() # IPTW

print(causal.estimates)


Treatment Effect Estimates: Weighting

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     10.002      0.002   6158.075      0.000      9.998     10.005



/Users/kai/anaconda3/lib/python3.10/site-packages/causalinference/estimators/weighting.py:23: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  wlscoef = np.linalg.lstsq(Z_w, Y_w)[0]
